In [1]:
import sys
print(sys.version) # broai supports python3.11

3.11.12 (main, Apr  9 2025, 04:04:00) [Clang 20.1.0 ]


# Start

In [2]:
%load_ext autoreload
%autoreload 2

# Setup for the Test

In [3]:
from broai.prompt_management.core import Persona, Instructions, Example, Examples
from broai.llm_management.ollama import BedrockOllamaChat
from broai.prompt_management.core import PromptGenerator
from broai.agent_management.core import BroAgent
from pydantic import BaseModel, Field
from typing import List, Any

In [4]:
bedrock_model = BedrockOllamaChat()

In [5]:
class Joke(BaseModel):
    setup:str = Field(description="this is a setup for the joke")
    punchline:str = Field(description="this is a punchline of the joke")

class Jokes(BaseModel):
    jokes:List[Joke]

In [6]:
class InputFormat(BaseModel):
    message:str = Field(description="The user message")

# BroAgent with full Framework: Happy Path

In [7]:
%%time
pg = PromptGenerator(
    persona="You are the good bro Andy.",
    instructions=Instructions(
        instructions=[
            "tell some jokes based on message",
        ],
    ),
    structured_output=Jokes,
    examples=Examples(examples=[
        Example(
            setting="Funny Andy",
            input=InputFormat(message="Gimme three jokes"),
            output=Jokes(jokes=[
                Joke(setup="the setup of the joke to build curiosity", punchline="the punchline is to complete the joke")
            ]),
        )
    ]),
    fallback=Jokes(jokes=[Joke(setup="error", punchline="error")])
)

bro = BroAgent(
    prompt_generator=pg,
    model=bedrock_model
)

response = bro.run(request=InputFormat(message="Tell me ten jokes."))
print(response)

jokes=[Joke(setup="Why don't scientists trust atoms?", punchline='Because they make up everything'), Joke(setup="Why don't eggs tell jokes?", punchline="They'd crack each other up"), Joke(setup='Why did the tomato turn red?', punchline='Because it saw the salad dressing'), Joke(setup='What do you call a fake noodle?', punchline='An impasta'), Joke(setup='Why did the scarecrow win an award?', punchline='Because he was outstanding in his field'), Joke(setup="Why don't lobsters share?", punchline="Because they're shellfish"), Joke(setup="What do you call a can opener that doesn't work?", punchline="A can't opener"), Joke(setup='I told my wife she was drawing her eyebrows too high.', punchline='She looked surprised'), Joke(setup="Why don't some couples go to the gym?", punchline="Because some relationships don't work out"), Joke(setup='Why did the bicycle fall over?', punchline='Because it was two-tired')]
CPU times: user 124 ms, sys: 13.8 ms, total: 138 ms
Wall time: 3.51 s


# BroAgent Full Framework: Pydantic Fallback

In [8]:
class Decoy(BaseModel):
    a:str
    b:str
    c:str
    e:int
    f:float

In [9]:
%%time
pg = PromptGenerator(
    persona="You are the good bro Andy.",
    instructions=Instructions(
        instructions=[
            "tell some jokes based on message",
        ],
    ),
    structured_output=Decoy,
    examples=Examples(examples=[
        Example(
            setting="Funny Andy",
            input=InputFormat(message="Gimme three jokes"),
            output=Jokes(jokes=[
                Joke(setup="the setup of the joke to build curiosity", punchline="the punchline is to complete the joke")
            ]),
        )
    ]),
    fallback=Jokes(jokes=[Joke(setup="error", punchline="error")])
)

bro = BroAgent(
    prompt_generator=pg,
    model=bedrock_model
)

response = bro.run(request="Tell me ten jokes.")
print(response)

jokes=[Joke(setup='error', punchline='error')]
CPU times: user 121 ms, sys: 0 ns, total: 121 ms
Wall time: 12.4 s


# BroAgent Full Framework: Default Fallback

In [10]:
%%time
pg = PromptGenerator(
    persona="You are the good bro Andy.",
    instructions=Instructions(
        instructions=[
            "tell some jokes based on message",
        ],
    ),
    structured_output=Decoy,
    examples=Examples(examples=[
        Example(
            setting="Funny Andy",
            input=InputFormat(message="Gimme three jokes"),
            output=Jokes(jokes=[
                Joke(setup="the setup of the joke to build curiosity", punchline="the punchline is to complete the joke")
            ]),
        )
    ]),
)

bro = BroAgent(
    prompt_generator=pg,
    model=bedrock_model
)

response = bro.run(request="Tell me ten jokes.")
print(response)

unknown error
CPU times: user 122 ms, sys: 0 ns, total: 122 ms
Wall time: 12.4 s


# BroAgent Full Framework: Custom Fallback in string

In [11]:
%%time
pg = PromptGenerator(
    persona="You are the good bro Andy.",
    instructions=Instructions(
        instructions=[
            "tell some jokes based on message",
        ],
    ),
    structured_output=Decoy,
    examples=Examples(examples=[
        Example(
            setting="Funny Andy",
            input=InputFormat(message="Gimme three jokes"),
            output=Jokes(jokes=[
                Joke(setup="the setup of the joke to build curiosity", punchline="the punchline is to complete the joke")
            ]),
        )
    ]),
    fallback="This is a custom fallback in string"
)

bro = BroAgent(
    prompt_generator=pg,
    model=bedrock_model
)

response = bro.run(request="Tell me ten jokes.")
print(response)

This is a custom fallback in string
CPU times: user 120 ms, sys: 162 μs, total: 120 ms
Wall time: 12.5 s


# BroAgent Full Framework: Custom Fallback not string

In [12]:
%%time
pg = PromptGenerator(
    persona="You are the good bro Andy.",
    instructions=Instructions(
        instructions=[
            "tell some jokes based on message",
        ],
    ),
    structured_output=Decoy,
    examples=Examples(examples=[
        Example(
            setting="Funny Andy",
            input=InputFormat(message="Gimme three jokes"),
            output=Jokes(jokes=[
                Joke(setup="the setup of the joke to build curiosity", punchline="the punchline is to complete the joke")
            ]),
        )
    ]),
    fallback={"error": "This is another custom fallback"}
)

bro = BroAgent(
    prompt_generator=pg,
    model=bedrock_model
)

response = bro.run(request="Tell me ten jokes.")
print(response)

{'error': 'This is another custom fallback'}
CPU times: user 120 ms, sys: 0 ns, total: 120 ms
Wall time: 12.5 s


# BroAgent with String: Happy Path

In [13]:
%%time
pg = PromptGenerator(
    persona="You are the good bro Andy.",
    instructions="tell some jokes based on message",
    structured_output="SETUP: \nthe setup of the joke to build curiosity\n\nPUNCHLINE: \nthe punchline is to complete the joke"
)

bro = BroAgent(
    prompt_generator=pg,
    model=bedrock_model
)

response = bro.run(request="Tell me ten jokes.")
print(response)

I gotcha! Here are ten jokes for ya:

1. Setup: I told my wife she was drawing her eyebrows too high.
Punchline: She looked surprised.

2. Setup: Why don't scientists trust atoms?
Punchline: Because they make up everything.

3. Setup: Why don't eggs tell jokes?
Punchline: They'd crack each other up.

4. Setup: What do you call a fake noodle?
Punchline: An impasta.

5. Setup: Why did the scarecrow win an award?
Punchline: Because he was outstanding in his field.

6. Setup: What do you call a can opener that doesn't work?
Punchline: A can't opener.

7. Setup: I'm reading a book about anti-gravity.
Punchline: It's impossible to put down.

8. Setup: Why did the bicycle fall over?
Punchline: Because it was two-tired.

9. Setup: What do you call a bear with no socks on?
Punchline: Barefoot.

10. Setup: Why did the banana go to the doctor?
Punchline: Because he wasn't peeling well.
CPU times: user 39.1 ms, sys: 0 ns, total: 39.1 ms
Wall time: 2.37 s


In [14]:
joke_in_string = response

# Bonus: Extract from normal string

In [15]:
# %%time

# class InputFormat(BaseModel):
#     content:str

# pg = PromptGenerator(
#     persona="You are a content extractor.",
#     instructions=Instructions(
#         instructions=[
#             "Extract the content into the sepcified JSON formant.",
#         ],
#     ),
#     structured_output=Jokes,
#     # examples=Examples(examples=[
#     #     Example(
#     #         setting="Joke Extraction",
#     #         input=InputFormat(content=joke_in_string),
#     #         output=Jokes(jokes=[
#     #             Joke(setup="the setup of the joke to build curiosity", punchline="the punchline is to complete the joke")
#     #         ]),
#     #     )
#     # ]),
#     fallback=Jokes(jokes=[Joke(setup="error", punchline="error")])
# )

# bro = BroAgent(
#     prompt_generator=pg,
#     model=bedrock_model
# )

# response = bro.run(request=InputFormat(content=joke_in_string))
# print(response)

# DuckStore

In [16]:
from broai.duckdb_management.utils import get_create_table_query, get_insert_query, get_batch_update_query
from broai.duckdb_management.interface import DuckStoreInterface

In [17]:
schemas = {
    "doc_id": "VARCHAR",
    "content": "VARCHAR",
    "data": "JSON"
}

sm = DuckStoreInterface(db="./duckmemory.db", table="sessionmemory", schemas=schemas)

In [18]:
sm.create_table()

In [19]:
sm.sql_df(query="SELECT * FROM sessionmemory;")

,doc_id,content,data


In [20]:
sm.show_schemas()

{'doc_id': 'VARCHAR', 'content': 'VARCHAR', 'data': 'JSON'}

In [21]:
_data = [
    ["0", "a"],
    ["1", "b"]
]
data = ", ".join([f"('{d[0]}', '{d[1]}')" for d in _data])
sm.add(fields=["doc_id", "content"], data=data)
sm.read(fields=["*"])

,doc_id,content,data
0,0,a,None
1,1,b,None


In [22]:
_data = [
    ["0", "aa"],
    ["1", "bb"]
]
data = ", ".join([f"('{d[0]}', '{d[1]}')" for d in _data])
sm.update(schemas={"doc_id": "VARCHAR", "content": "VARCHAR"}, data=data, ref_keys=["doc_id"])
sm.read()

,doc_id,content,data
0,0,aa,None
1,1,bb,None


In [23]:
sm.delete(where_condition="WHERE doc_id IN ('1', '2')")
sm.read()

,doc_id,content,data
0,0,aa,None


In [24]:
sm.delete_table()

In [25]:
sm.drop_table()

In [26]:
sm.remove_database(confirm="remove database")

# Utility

In [27]:
from broai.interface import Context, Contexts, TaskStatus

In [28]:
Context(context="Test")

Context(id='3a419487-ffc3-47c3-98a7-4e6cdf13417a', context='Test', metadata=None, type='document', created_at='2025-04-21 23:41:52.356769')

In [29]:
contexts = Contexts(contexts=[Context(context=c, metadata={"source": s}) for c, s in zip(["test1", "test2"], ["source1", "source2"])])

In [30]:
print(contexts.as_knowledge())

Knowledge: 

Source: source1
Context: 
test1

Source: source2
Context: 
test2


# Experiment

## pdf_to_markdown

In [31]:
from broai.experiments.pdf_to_markdown import pdf_to_markdown

markdown_text, images = pdf_to_markdown("./docs/test1/storm.pdf")

/home/ec2-user/SageMaker/broai/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_18850/2694963727.py:3: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: pdf_to_markdown
  markdown_text, images = pdf_to_markdown("./docs/test1/storm.pdf")


Loaded layout model s3://layout/2025_02_18 on device cuda with dtype torch.float16


Loaded texify model s3://texify/2025_02_18 on device cuda with dtype torch.float16


Loaded recognition model s3://text_recognition/2025_02_18 on device cuda with dtype torch.float16


Loaded table recognition model s3://table_recognition/2025_02_18 on device cuda with dtype torch.float16


Loaded detection model s3://text_detection/2025_02_28 on device cuda with dtype torch.float16


Loaded detection model s3://inline_math_detection/2025_02_24 on device cuda with dtype torch.float16


Running OCR Error Detection: 100%|██████████| 7/7 [00:00<00:00, 53.59it/s]
Detecting bboxes: 0it [00:00, ?it/s]
Texify inference: 100%|██████████| 1/1 [00:01<00:00,  1.49s/it]
Detecting bboxes: 0it [00:00, ?it/s]
Recognizing tables: 100%|██████████| 2/2 [00:01<00:00,  1.40it/s]


In [32]:
with open("./docs/test1/storm.md", "w") as f:
    f.write(markdown_text)

## chunk

In [33]:
with open("./docs/test1/storm.md", "r") as f:
    markdown_text = f.read()

In [34]:
from broai.experiments.chunk import split_markdown, consolidate_markdown, get_markdown_sections, split_overlap, chunk_chunks
from broai.interface import Context, Contexts

In [35]:
chunks = split_markdown(markdown_text)

Markdown headings: max(4)


/tmp/ipykernel_18850/3854812085.py:1: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: split_markdown
  chunks = split_markdown(markdown_text)


In [36]:
len(chunks)

54

In [37]:
consolidated_chunks = consolidate_markdown(chunks)
len(consolidated_chunks)

/tmp/ipykernel_18850/169284658.py:1: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: consolidate_markdown
  consolidated_chunks = consolidate_markdown(chunks)


50

In [38]:
sections = get_markdown_sections(consolidated_chunks)
len(sections)

/tmp/ipykernel_18850/2669657907.py:1: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: get_markdown_sections
  sections = get_markdown_sections(consolidated_chunks)


50

In [39]:
contexts = Contexts()
source = ".docs/test1/storm.md"
for section, chunk in zip(sections, consolidated_chunks):
    contexts.add_context(Context(context=chunk, metadata={"section": section, "source": source, "type": "document"}))
len(contexts.contexts)

50

In [40]:
new_contexts = split_overlap(contexts.contexts)
new_contexts = Contexts(contexts=new_contexts)
len(new_contexts.contexts)

/tmp/ipykernel_18850/1421024119.py:1: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: split_overlap
  new_contexts = split_overlap(contexts.contexts)


85

In [41]:
new_contexts.contexts[:3]

[Context(id='e41a46f0-c213-4624-bbe3-831e29c6546d', context='# arXiv:2402.14207v2 [cs.CL] 8 Apr 2024\n\nAssisting in Writing Wikipedia-like Articles From Scratch with Large Language Models\n\nYijia Shao Yucheng Jiang Theodore A. Kanell Peter Xu Omar Khattab Monica S. Lam\n\nStanford University\n\n{shaoyj, yuchengj, tkanell, peterxu, okhattab}@stanford.edu lam@cs.stanford.edu\n', metadata={'section': '# arXiv:2402.14207v2 [cs.CL] 8 Apr 2024', 'source': '.docs/test1/storm.md', 'type': 'document', 'sequence': 0}, type='document', created_at='2025-04-21 23:43:05.567994'),
 Context(id='8e114076-b758-4e1d-abfe-0ee0f001b4b4', context="Abstract\n\nWe study how to apply large language models to write grounded and organized long-form articles from scratch, with comparable breadth and depth to Wikipedia pages. This underexplored problem poses new challenges at the *pre-writing* stage, including how to research the topic and prepare an outline prior to writing. We propose STORM, a writing system f

In [42]:
chunk_chunks([c.context for c in new_contexts.contexts])

[0] | tokens: 35 | chars: 309
[1] | tokens: 189 | chars: 1349
[2] | tokens: 500 | chars: 4170
[3] | tokens: 462 | chars: 3550
[4] | tokens: 500 | chars: 2030
[5] | tokens: 211 | chars: 1257
[6] | tokens: 162 | chars: 1096
[7] | tokens: 238 | chars: 1830
[8] | tokens: 196 | chars: 2327
[9] | tokens: 256 | chars: 1831
[10] | tokens: 226 | chars: 1627
[11] | tokens: 114 | chars: 759
[12] | tokens: 150 | chars: 1002
[13] | tokens: 57 | chars: 422
[14] | tokens: 166 | chars: 1305
[15] | tokens: 105 | chars: 681
[16] | tokens: 500 | chars: 1605
[17] | tokens: 500 | chars: 1131
[18] | tokens: 500 | chars: 868
[19] | tokens: 480 | chars: 1431
[20] | tokens: 500 | chars: 3018
[21] | tokens: 320 | chars: 924
[22] | tokens: 196 | chars: 1381
[23] | tokens: 500 | chars: 2242
[24] | tokens: 500 | chars: 3699
[25] | tokens: 260 | chars: 1890
[26] | tokens: 441 | chars: 3970
[27] | tokens: 88 | chars: 637
[28] | tokens: 159 | chars: 1194
[29] | tokens: 62 | chars: 413
[30] | tokens: 230 | chars: 1487

## Enmedding: BAAI/bge-m3

In [43]:
from broai.experiments.huggingface_embedding import BAAIEmbedding, EmbeddingDimension

In [44]:
EmbeddingDimension.BAAI_BGE_M3.value

1024

In [45]:
baai_em = BAAIEmbedding()

/tmp/ipykernel_18850/107190707.py:1: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: BAAIEmbedding
  baai_em = BAAIEmbedding()
Fetching 30 files: 100%|██████████| 30/30 [00:08<00:00,  3.55it/s]


In [46]:
sentences_1 = ["What is BGE M3?", "Defination of BM25"]
sentences_2 = ["BGE M3 is an embedding model supporting dense retrieval, lexical matching and multi-vector interaction.", 
               "BM25 is a bag-of-words retrieval function that ranks a set of documents based on the query terms appearing in each document"]

In [47]:
baai_em.run(sentences_1)

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


array([[-0.03412  , -0.047    , -0.0009174, ...,  0.04828  ,  0.00756  ,
        -0.0296   ],
       [-0.010376 , -0.04483  , -0.02428  , ..., -0.00822  ,  0.01502  ,
         0.011086 ]], shape=(2, 1024), dtype=float16)

## CrossEncoder: cross-encoder/ms-marco-MiniLM-L6-v2

In [48]:
from broai.experiments.cross_encoder import ReRanker
from broai.interface import Context

In [49]:
rr = ReRanker()

/tmp/ipykernel_18850/3697317191.py:1: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: ReRanker
  rr = ReRanker()


In [50]:
query = "pandas is good"
contexts = [Context(context=con, metadata={"source":"test"}) for con in ["pandas is goose", "pandas is good", "pandas is great", "pandas is goat", "pandas is gang"]]

In [51]:
reranked_contexts, scores = rr.run(query, contexts, top_n=3)
reranked_contexts

[Context(id='64225d55-6a8f-4150-93d9-6b8870ae8f17', context='pandas is good', metadata={'source': 'test'}, type='document', created_at='2025-04-21 23:43:26.513680'),
 Context(id='40358aa6-9779-4405-b937-af74e488c0f6', context='pandas is great', metadata={'source': 'test'}, type='document', created_at='2025-04-21 23:43:26.513698'),
 Context(id='5e2e901b-6dcf-4e20-a040-1349c53b5597', context='pandas is goat', metadata={'source': 'test'}, type='document', created_at='2025-04-21 23:43:26.513710')]

In [52]:
scores

[8.630863189697266, 7.362998962402344, 0.6360796689987183]

## ExpBroAgent

In [53]:
from broai.experiments.bro_agent import BroAgent

In [54]:
%%time
pg = PromptGenerator(
    persona=Persona(name="Bro Andy", description="You are the best bro who's cool and supportive."),
    instructions=Instructions(
        instructions=[
            "tell some jokes based on message",
        ],
    ),
    structured_output=Jokes,
    examples=Examples(examples=[
        Example(
            setting="Funny Andy",
            input=InputFormat(message="Gimme three jokes"),
            output=Jokes(jokes=[
                Joke(setup="the setup of the joke to build curiosity", punchline="the punchline is to complete the joke")
            ]),
        )
    ]),
    fallback=Jokes(jokes=[Joke(setup="error", punchline="error")])
)

bro = BroAgent(
    prompt_generator=pg,
    model=bedrock_model
)

response = bro.run(request=InputFormat(message="Tell me twenty jokes."))
print(response)

jokes=[Joke(setup="Why don't scientists trust atoms?", punchline='Because they make up everything'), Joke(setup="Why don't eggs tell jokes?", punchline="They'd crack each other up"), Joke(setup='Why did the tomato turn red?', punchline='Because it saw the salad dressing'), Joke(setup='What do you call a fake noodle?', punchline='An impasta'), Joke(setup='Why did the scarecrow win an award?', punchline='Because he was outstanding in his field'), Joke(setup="Why don't lobsters share?", punchline="Because they're shellfish"), Joke(setup="What do you call a can opener that doesn't work?", punchline="A can't opener"), Joke(setup='I told my wife she was drawing her eyebrows too high.', punchline='She looked surprised'), Joke(setup="Why don't some couples go to the gym?", punchline="Because some relationships don't work out"), Joke(setup='Why did the bicycle fall over?', punchline='Because it was two-tired'), Joke(setup='What do you call a bear with no socks on?', punchline='Barefoot'), Joke(

In [55]:
for enum, j in enumerate(response.jokes):
    print("Joke:", enum+1)
    print("setup:", j.setup)
    print("punchline:", j.punchline)
    print("="*20)

Joke: 1
setup: Why don't scientists trust atoms?
punchline: Because they make up everything
Joke: 2
setup: Why don't eggs tell jokes?
punchline: They'd crack each other up
Joke: 3
setup: Why did the tomato turn red?
punchline: Because it saw the salad dressing
Joke: 4
setup: What do you call a fake noodle?
punchline: An impasta
Joke: 5
setup: Why did the scarecrow win an award?
punchline: Because he was outstanding in his field
Joke: 6
setup: Why don't lobsters share?
punchline: Because they're shellfish
Joke: 7
setup: What do you call a can opener that doesn't work?
punchline: A can't opener
Joke: 8
setup: I told my wife she was drawing her eyebrows too high.
punchline: She looked surprised
Joke: 9
setup: Why don't some couples go to the gym?
punchline: Because some relationships don't work out
Joke: 10
setup: Why did the bicycle fall over?
punchline: Because it was two-tired
Joke: 11
setup: What do you call a bear with no socks on?
punchline: Barefoot
Joke: 12
setup: Why did the bana